### How would the annual performance reports of honey bees compare to the annual reports released by Bumble? What are the relative strengths and weaknesses of each entity and what opportunities and risks should potential stakeholders be aware of? 
#### Key Metrics (annually and 3 year average)
    Revenue
	EBITDA and EBITDA Margin
    Metrics above at a per user level
	Main areas of financial loss
#### Other Questions
-How would queen bees be compensated if they were paid on similar scale to bumble CEOs (ceo comp data is in bumble proxy report)

-How much are the relative workforce costs?

-Can we track any seasonality trend (seasonality data is reported in bumble annual report)

In [2]:
#Library setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import locale
locale.setlocale(locale.LC_NUMERIC, 'en_US.UTF-8')

'en_US.UTF-8'

### Bumble Inc Metrics

In [4]:
#Read in and clean annual report
bb_op_costs23 = pd.read_csv('../data/23operating_costs.csv')
bb_op_costs = bb_op_costs23.rename(columns = {'in_thousands': 'metric_in_thousands', 'ye_2023': '2023', 'ye_2022': '2022', 'ye_2021': '2021'})
bb_op_costs = bb_op_costs.set_index('metric_in_thousands').transpose()
bb_op_costs = bb_op_costs.reset_index()
bb_op_costs = bb_op_costs.reset_index(drop = True).rename(columns={'index': 'Year'})
bb_op_costs.head(3)


metric_in_thousands,Year,Revenue,Costofrevenue,Sellingandmarketingexpense,Generalandadministrativeexpense,Productdevelopmentexpense,Depreciationandamortizationexpense,Totaloperatingcostsandexpenses,Operatingearnings(loss),"Interestincome(expense),net","Otherincome(expense),net",Income(loss)beforeincometax,Incometaxbenefit(provision),Netearnings(loss),Netearnings(loss)attributabletononcontrollinginterests,Netearnings(loss)attributabletoBumbleInc.shareholders
0,2023,1051830,307835,270380,221649,130565,68028,998457,53373,-21534,-26537,5302,-7170,-1868,2345,-4213
1,2022,"903,503","249,490","249,269","308,855","109,020","89,713","1,006,347",-102844,"-24,063",16189,-110718,-3406,-114124,-34378,-79746
2,2021,"760,910","205,573","211,711","257,489","113,764","107,056","895,593","(134,683","-24,574","3,160",-156097,"437,837","281,740",-28075,"309,815"


##### Revenue

In [6]:
#Format revenue values with locale to remove commas and cast as float
bb_op_costs['Revenue'] = bb_op_costs['Revenue'].astype(str)
rev_list = []
for value in bb_op_costs['Revenue']:
    rev_list.append(locale.atof(value))
bb_op_costs['Revenue'] = rev_list

In [7]:
#Revenue: 3yr Average and Year-over-Year
rev_3yr_avg = bb_op_costs['Revenue'].mean()
rev_yoy = bb_op_costs[['Year','Revenue']]


##### Ebitda

In [9]:
#Format total operating costs and expenses with locale to remove commas and cast as float
bb_op_costs['Totaloperatingcostsandexpenses'] = bb_op_costs['Totaloperatingcostsandexpenses'].astype(str)
topce_list = []
for value in bb_op_costs['Totaloperatingcostsandexpenses']:
    topce_list.append(locale.atof(value))
bb_op_costs['Totaloperatingcostsandexpenses'] = topce_list

In [10]:
#Format depreciation and amortization expenses with locale to remove commas and cast as float
bb_op_costs['Depreciationandamortizationexpense'] = bb_op_costs['Depreciationandamortizationexpense'].astype(str)
daa_list = []
for value in bb_op_costs['Depreciationandamortizationexpense']:
    daa_list.append(locale.atof(value))
bb_op_costs['Depreciationandamortizationexpense'] = daa_list

In [11]:
#Ebitda: 3yr Average and Year-over-Year
bb_op_costs['Ebitda_Earnings'] = bb_op_costs['Revenue'] - bb_op_costs['Totaloperatingcostsandexpenses'] + bb_op_costs['Depreciationandamortizationexpense']
ebitda_3yr_avg = bb_op_costs['Ebitda_Earnings'].mean()
ebitda_yoy = bb_op_costs[['Year','Ebitda_Earnings']]

#Ebitda Margin: 3yr Average and Year-over-Year
bb_op_costs['Ebitda_Margin'] = ((bb_op_costs['Ebitda_Earnings']/bb_op_costs['Revenue'])*100).round(0)
ebitda_margin_3yr_avg = (((bb_op_costs['Ebitda_Earnings'].sum())/(bb_op_costs['Revenue'].sum()))*100).round(0) #is this the best way to calculate this?
ebitda_margin_yoy = bb_op_costs[['Year','Ebitda_Margin']]

##### Revenue Per User

##### Ebitda Earnings Per User

##### Highest Area of Loss

### Honey Bee Metrics

In [16]:
#Read in and clean USDA honey info
honey23 = pd.read_csv('../data/23Nasshoney.csv')
honey22 = pd.read_csv('../data/22Nasshoney.csv')
honey21 = pd.read_csv('../data/21Nasshoney.csv')
honey20 = pd.read_csv('../data/20Nasshoney.csv')
inc_exp = pd.read_csv('../data/2223Nass_inc_and_exp.csv')
honey = pd.concat([honey20,honey21, honey22, honey23])

In [17]:
#honey sold = stock at end of year prior + year production + minus stock at end of current year
#revenue = honey sold * avg 

##### Revenue

Honey Sales

In [68]:
# Calculate annual honey sales by determing YoY depletion of stockpiles
honey['year_prior'] = honey['year'] - 1
honey_sales = pd.merge(honey, honey, left_on = ['state','year_prior'], right_on = ['state','year'])
honey_sales = honey_sales[['state', 'production_1000_pounds_x','stock_1000_pounds_x','price_per_pounds_dollars_x', 'year_x', 'year_prior_x', 'stock_1000_pounds_y']]
honey_sales = honey_sales.rename(columns = { 'production_1000_pounds_x': 'production_1000lbs_cy','stock_1000_pounds_x':'stock_1000lbs_cy','price_per_pounds_dollars_x':'price_per_lbs_cy', 'year_x':'current_year', 'year_prior_x':'year_prior', 'stock_1000_pounds_y':'stock_1000lbs_yp'})

In [21]:
#Format production numbers using locale
honey_sales['production_1000lbs_cy'] = honey_sales['production_1000lbs_cy'].astype(str)
pcy_list = []
for value in honey_sales['production_1000lbs_cy']:
    pcy_list.append(locale.atof(value))
honey_sales['production_1000lbs_cy'] = pcy_list

In [22]:
#Format stock numbers (current year) using locale
honey_sales['stock_1000lbs_cy'] = honey_sales['stock_1000lbs_cy'].astype(str)
scy_list = []
for value in honey_sales['stock_1000lbs_cy']:
    scy_list.append(locale.atof(value))
honey_sales['stock_1000lbs_cy'] = scy_list

In [23]:
#Format stock numbers (prior year) using locale
honey_sales['stock_1000lbs_yp'] = honey_sales['stock_1000lbs_yp'].astype(str)
spy_list = []
for value in honey_sales['stock_1000lbs_yp']:
    spy_list.append(locale.atof(value))
honey_sales['stock_1000lbs_yp'] = spy_list

In [66]:
#calculate annual sales (in 1000 pounds and in 1000 dollars)
honey_sales['sales_1000lbs'] = (honey_sales['stock_1000lbs_yp'] + honey_sales['production_1000lbs_cy'] - honey_sales['stock_1000lbs_cy'])
honey_sales['sales_1000dollars'] = honey_sales['sales_1000lbs'] * honey_sales['price_per_lbs_cy']